<a href="https://colab.research.google.com/github/waveology/aire/blob/main/4_caracterizaci%C3%B3n_estadistica_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caracterización estadística de los datos

Vamos a explorar algunas posibilidades de análisis de datos de calidad del aire que ofrece Python.

Para simplificar la tarea hemos puesto las funciones de lectura de ficheros descritas anteriormente en otro notebook que descargamos desde el repositorio github

In [ ]:
# Borramos copias previas y clonamos el repositorio 
# ------------------------------------------------------------
!rm -rf aire && git clone https://github.com/waveology/aire.git
%cd aire

Instalamos la extensión que nos permite importar notebooks:

In [ ]:
!pip install import-ipynb

Importamos las extensiones que vamos a necesitar:

In [ ]:
import import_ipynb               # permite importar código Python de otros Notebooks
import lectura_de_datos           # lee ficheros de datos meteorológicos y de contaminación de Madrid
import matplotlib.pyplot as plt   # dibujo de gráficos
from scipy import stats           # cálculo estadístico

In [ ]:
import lee_datos

**Código | Municipio | Nombre**

---
*       28005002   :    (  5,  'ALCALÁ DE HENARES'), 
*       28006004   :    (  6,  'ALCOBENDAS'), 
*       28007004   :    (  7,  'ALCORCÓN'), 
*       28009001   :	(  9,  'ALGETE'), 
*       28013002   :	( 13,  'ARANJUEZ'), 
*       28014002   :	( 14,  'ARGANDA DEL REY'),
*       28016001   :	( 16,  'EL ATAZAR'),
*       28045002   :	( 45,  'COLMENAR VIEJO'), 
*       28047002   :	( 47,  'COLLADO VILLALBA'), 
*       28049003   :	( 49,  'COSLADA'), 
*       28058004   :	( 58,  'FUENLABRADA'), 
*       28065014   :	( 65,  'GETAFE'), 
*       28067001   :	( 67,  'GUADALIX DE LA SIERRA'), 
*       28074007   :	( 74,  'LEGANÉS'), 
*       28080003   :	( 80,  'MAJADAHONDA'), 
*       28092005   :	( 92,  'MÓSTOLES'), 
*       28102001   :	(102,  'ORUSCO DE TAJUÑA'), 
*       28120001   : 	(120,  'PUERTO DE COTOS'), 
*       28123002   :	(123,  'RIVAS-VACIAMADRID'), 
*       28133002   :	(133,  'SAN MARTÍN DE VALDEIGLESIAS'), 
*       28148004   :	(148,  'TORREJÓN DE ARDOZ'), 
*       28161001   :	(161,  'VALDEMORO'), 
*       28171001   :	(171,  'VILLA DEL PRADO'), 
*       28180001   :	(180,  'VILLAREJO DE SALVANÉS')


**Código  |  Magnitud  | Unidades**

  
---
*      1 	:  ('Dióxido de azufre', 'μg/m³'),
*      6 	:  ('Monóxido de carbono', 'mg/m³'),
*      7 	:  ('Monóxido de nitrógeno', 'μg/m³'),
*      8 	:  ('Dióxido de nitrógeno', 'μg/m³'),
*      9 	:  ('Partículas en suspensión < PM2.5', 'μg/m³'),
*     10 	:  ('Partículas en suspensión < PM10',  'μg/m³'),
*     12 	:  ('Óxidos de nitrógeno', 'μg/m³'),
*     14 	:  ('Ozono', 'μg/m³'),
*     20 	:  ('Tolueno', 'μg/m³'),
*     22 	:  ('Black Carbon', 'μg/m³'),
*     30 	:  ('Benceno', 'μg/m³'),
*     42 	:  ('Hidrocarburos totales', 'mg/m³'),
*     44 	:  ('Hidrocarburos no metánicos', 'mg/m³'),
*    431  :  ('MetaParaXileno', 'μg/m³')
    }

**Código | Magnitud | Unidades**

---
*   81 :	('Velocidad del viento',     'm/s'), 
*   82 :	('Dirección del viento',     'º'), 
*   83 :	('Temperatura',              'ºC'), 
*   86 :	('Humedad relativa',         '%'), 
*   87 :	('Presión atmosférica',      'hPa'), 
*   88 :	('Radiación solar',          'W/m2'), 
*   89 :	('Precipitación',            'mm')

In [ ]:
codigo_magnitud = 83         # Temperatura

codigo_estacion=28067001     # Guadalix de la Sierra

yearlist = ('2020','2021','2022')

In [ ]:
df = []
for year in yearlist :
    
    d, magnitud, unidades,estacion = lectura_de_datos.meteo(
        
             'datos/meteo/%s.csv' % year,

              codigo_magnitud = codigo_magnitud, 

              codigo_estacion = codigo_estacion) 
    
    df.append(d)